# DATA EXTRACTION AND NLP PROJECT

## Dataset Link

Input Link:- https://docs.google.com/spreadsheets/d/1D7QkDHxUSKnQhR--q0BAwKMxQlUyoJTQ/edit?usp=drive_link&ouid=108072191480817696797&rtpof=true&sd=true

Stop Words:- https://drive.google.com/file/d/1aWxyJI0d9MOk59OZ_unfBY5E-Nvg_ezW/view?usp=drive_link,

https://drive.google.com/file/d/1K-6MjPq5AQg4ICYY6PDfapB7JECUnryD/view?usp=drive_link,

https://drive.google.com/file/d/13LXnH6vaJhvY4s2ai_2oW2qwongU_iAI/view?usp=drive_link,

 https://drive.google.com/file/d/1tTDfLXNPxNuUGZXHQkQhW6wPf4Xnivwr/view?usp=drive_link,

 https://drive.google.com/file/d/1PnZhcsfjBVxnzwa4N6MrLWf6Kuhhjpdk/view?usp=drive_link,

https://drive.google.com/file/d/1RKxMOHzBdLrGuYb7MCJRTKKPwDG9Agbe/view?usp=drive_link,
  
 https://drive.google.com/file/d/1mBOuggD8AVNFjr9sprLoD2_6mVWAgRGE/view?usp=drive_link

Positive Negative words:- https://drive.google.com/file/d/1qqMwc_-ayS38HEOB97osO_nkIxRkbnvh/view?usp=drive_link,

https://drive.google.com/file/d/1seAj8G42SmfgUUx8lqVDJofm4Tuh2TOT/view?usp=drive_link


### IMPORTANT LIBRARIES

In [1]:
import re
import io
import nltk
import os, sys
import requests
import string, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from textblob import TextBlob
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.classify import NaiveBayesClassifier,accuracy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV



### Preprocessing

In [2]:
File = pd.read_excel("/content/Input.xlsx")

In [3]:
# For not losing original dataset
Data = File.copy()

In [4]:
Data.head()

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...


In [5]:
Data.shape

(114, 2)

In [6]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   URL_ID  114 non-null    float64
 1   URL     114 non-null    object 
dtypes: float64(1), object(1)
memory usage: 1.9+ KB


Scrap data from Urls

In [7]:
%%time
data = []
for i in range(0,6):
  url = File['URL'][i]
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser')

  title= soup.title.text.replace("\n", " ")
  text= soup.text.replace("\n", " ")
  data.append([title,text])

CPU times: user 682 ms, sys: 24.5 ms, total: 706 ms
Wall time: 3.37 s


In [8]:
df1 = pd.DataFrame(data, columns= ['Title','Text'])

In [9]:
df1.head()

,Title,Text
0,Rise of telemedicine and its Impact on Livelih...,Rise of telemedicine and its Impact on ...
1,Rise of e-health and its impact on humans by t...,Rise of e-health and its impact on huma...
2,Rise of e-health and its impact on humans by t...,Rise of e-health and its impact on huma...
3,Rise of telemedicine and its Impact on Livelih...,Rise of telemedicine and its Impact on ...
4,Rise of telemedicine and its Impact on Livelih...,Rise of telemedicine and its Impact on ...


I choose only 6 row for better understaning and reduce time complexcity if you want to chhose more row then just change in range in df1 and slicing in demo dataframe

In [10]:
demo = Data.loc[0:5]

In [11]:
demo.head()

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...


In [12]:
df=pd.concat([demo,df1], axis=1)

In [13]:
df.head()

,URL_ID,URL,Title,Text
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,Rise of telemedicine and its Impact on ...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,Rise of e-health and its impact on huma...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,Rise of e-health and its impact on huma...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,Rise of telemedicine and its Impact on ...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,Rise of telemedicine and its Impact on ...


In [14]:
df.shape

(6, 4)

In [15]:
df['Text'][1]

'       Rise of e-health and its impact on humans by the year 2030 - Blackcoffer Insights                                                                                            Sign in     Our Success Stories  Banking, Financials, Securities, and Insurance Energy Entertainment Fast Moving Consumer Goods Government & Think Tanks Healthcare Infrastructure & Real Estate IT Lifestyle, eCommerce & Online Market Place Production & Manufacturing Research & Academia Retail & Supply Chain Telecom   What We Do  Banking, Financials, Securities, and Insurance Energy Entertainment Fast Moving Consumer Goods Government & Think Tanks Healthcare Hospitality Infrastructure & Real Estate IT Services Lifestyle, eCommerce & Online Market Place News & Media Production & Manufacturing Research & Academia Retail & Supply Chain   What We Think  Automobiles & Components BFSI Asset and Portfolio Banks Capital Markets Derivatives and Securities Diversified Financials Finance & Accounting Insurance Securities

Converting to lower case

In [16]:
# df['Text'] = df['Text'].str.lower()

Remove Hyper-links


In [17]:
df['Text'] = df['Text'].str.replace(r"http\S+","", regex=True)

Remove Emoji

In [18]:
# Remove Emojis if needed
# def remove_emoji(string):
#     emoji_pattern = re.compile("["
#                            u"\U0001F600-\U0001F64F"  # emoticons
#                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
#                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                            u"\U00002702-\U000027B0"
#                            u"\U000024C2-\U0001F251"
#                            "]+", flags=re.UNICODE)
#     return emoji_pattern.sub(r'', string)

Remove Whitespace


In [19]:
def remove_whitespace(text):
    return  " ".join(text.split())

df["Text"]= df["Text"].apply(remove_whitespace)

Remove numbers

In [20]:
def remove_numbers(text):
    result = re.sub(r'\d+', '', text)
    return result

df["Text"]= df["Text"].apply(remove_numbers)

Spelling checker

In [21]:
df["Text"] = df["Text"].apply(lambda x: str(TextBlob(x)))

Remove Punctuation

In [22]:
def remove_punctuation(text):
    return re.sub(r'[^\w\s.]', '', text)

df['Text'] = df['Text'].apply(remove_punctuation)


Remove Stopwords

In [23]:
def removeStopWords(series, stopWords):
  data = []
  for series in series:
    words = series.split()
    filteredWords = [ word for word in words if word not in stopWords ]
    data.append(' '.join(filteredWords))

  return data

In [24]:
path = "/content/stopwords/"

dir  = os.listdir(path)

for file in dir:
  # print("file : " + file, end="\n")
  if(file.startswith("Stop")):
    with open(path+file,"r",encoding="ISO-8859-1") as f:
            data = f.read()
            stopWords = data.split('\n')

            df['Text'] = removeStopWords(df['Text'], stopWords)

Text Normalization

In [25]:
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
def lemmatize_words(text):
  words = text.split()
  words = [lemmatizer.lemmatize(word,pos='v') for word in words]
  return ' '.join(words)

In [27]:
df['Text'] = df['Text'].apply(lemmatize_words)

Tokenization

In [28]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [29]:
def tokens(text):
  nltk_tokens = nltk.word_tokenize(text)
  return nltk_tokens

df["tokens"] = df.Text.apply(tokens)


Count number of words in Text

In [30]:
def count_words(sentence):
    words = sentence.split()
    return len(words)

df['word_count'] = df['Text'].apply(count_words)

Polarity Score

In [31]:
def calculate_polarity(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity


df['Polarity'] = df['Text'].apply(calculate_polarity)


Subjectivity Score

In [32]:
def calculate_subjectivity(text):
    analysis = TextBlob(text)
    return analysis.sentiment.subjectivity

df['Subjectivity_Score'] = df['Text'].apply(calculate_subjectivity)
df.head()

,URL_ID,URL,Title,Text,tokens,word_count,Polarity,Subjectivity_Score
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,Rise telemedicine Impact Livelihood Blackcoffe...,"[Rise, telemedicine, Impact, Livelihood, Black...",1610,0.080363,0.371509
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,Rise ehealth impact humans year Blackcoffer In...,"[Rise, ehealth, impact, humans, year, Blackcof...",1005,0.106257,0.487765
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,Rise ehealth impact humans year Blackcoffer In...,"[Rise, ehealth, impact, humans, year, Blackcof...",1318,0.104491,0.400472
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,Rise telemedicine Impact Livelihood Blackcoffe...,"[Rise, telemedicine, Impact, Livelihood, Black...",1397,0.109602,0.367165
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,Rise telemedicine Impact Livelihood Blackcoffe...,"[Rise, telemedicine, Impact, Livelihood, Black...",1391,0.111983,0.362926


Open sentiment file

In [33]:
# For Negative Words
file_path = "/content/sentiment/negative-words.txt"

# Initialize an empty variable to store the file contents
neg_words = ""

with open(file_path, "r", encoding="ISO-8859-1") as file:
        neg_words = file.read()


In [34]:
# For Negative Words
file_path = "/content/sentiment/positive-words.txt"

# Initialize an empty variable to store the file contents
pos_words = ""

with open(file_path, "r", encoding="ISO-8859-1") as file:
        pos_words = file.read()


Positve Score

In [35]:

def calculate_positive_score(text):
    # Tokenize the input text into words
    words = text.split()

    # Initialize the Positive Score
    positive_score = 0

    # Calculate the Positive Score by summing the scores of positive words
    for word in words:
        if word in pos_words:
            positive_score += 1

    return positive_score



df['positive_socre'] = df['Text'].apply(calculate_positive_score)



Negative Score

In [36]:

def calculate_negative_score(text):
    words = text.split()

    # Initialize the negative score
    negative_score = 0

    # Check each word against the Negative Dictionary
    for word in words:
        if word in neg_words:
            negative_score -= 1  # Assign a value of -1 for each negative word found

    # Multiply the score by -1 to make it positive
    negative_score *= -1

    return negative_score

df['negative_socre'] = df['Text'].apply(calculate_negative_score)

AVG SENTENCE LENGTH

In [37]:
def calculate_average_sentence_length(dataset):
    # Split the dataset into sentences using regular expressions
    sentences = re.split(r'[.!?]', dataset)

    # Initialize variables to keep track of the total number of words and sentences
    total_words = 0
    total_sentences = len(sentences) - 1  # Subtract 1 to account for the last empty string

    # Iterate through the sentences to count the words in each sentence
    for sentence in sentences:
        words = sentence.split()
        total_words += len(words)

    # Calculate the average sentence length
    if total_sentences > 0:
        average_length = total_words / total_sentences
    else:
        average_length = 0

    return average_length

df["average_sentence_length"]= df['Text'].apply(calculate_average_sentence_length)

Average Word Length

In [38]:
def average_word_length(text):
    total_characters = 0
    total_words = 0

    words = text.split()

    for word in words:
        total_characters += len(word)
        total_words += 1

    if total_words > 0:
        average_length = total_characters / total_words
    else:
        average_length = 0

    return round(average_length)


df["Average_word_length"] = df["Text"].apply(average_word_length)

AVG NUMBER OF WORDS PER SENTENCE

In [39]:

def avg_words_per_sentence(text):
    sentences = nltk.sent_tokenize(text)
    num_sentences = len(sentences)
    num_words = sum(len(nltk.word_tokenize(sentence)) for sentence in sentences)
    if num_sentences > 0:
        return num_words / num_sentences
    else:
        return 0  # Avoid division by zero if there are no sentences

df['Avg_Words_Per_Sentence'] = df['Text'].apply(avg_words_per_sentence)


SYLLABLE PER WORD

In [40]:
def syllableCount(text: string) -> int:
  words = text.lower().split()
  totSyll = 0
  for word in words:
    for char in word:
      if(char in ['a','e','i','o','u']):
        totSyll = totSyll + 1
    if(word.endswith('es') or word.endswith('ed')):
      totSyll = totSyll - 1
  return totSyll

df['syllable_count'] = df['Text'].apply(syllableCount)


COMPLEX WORD COUNT

In [41]:
def complexWordCount(text: string) -> int:
  words = text.lower().split()
  count = 0
  for word in words:
    totalSyll = sum(1 for char in word if char in ['a','e','i','o','u'])
    if(word.endswith('es') or word.endswith('ed')):
      totalSyll = totalSyll - 1
    if(totalSyll > 2):
      count = count + 1
  return count

df['Complex_word_count']= df['Text'].apply(complexWordCount)

PERCENTAGE OF COMPLEX WORDS


In [42]:
df['percentage_of_complex_word']= df['Complex_word_count']/df['word_count']*100

FOG INDEX

In [43]:
df['Fog_index'] = 0.4 * (df['average_sentence_length'] + df['percentage_of_complex_word'])


PERSONAL PRONOUNS

In [44]:
def count_personal_pronouns(text):

    regex = r'\b(I|[Mm]y|[Oo]urs|[Ww]e|us)\b(?!S\b)'

    # Use re.findall to find all matches of the pattern in the text
    matches = re.findall(regex, text)

    count = len(matches)

    return count

df['Count_personal_pronouns'] = df["Text"].apply(count_personal_pronouns)

In [45]:
df.head()

,URL_ID,URL,Title,Text,tokens,word_count,Polarity,Subjectivity_Score,positive_socre,negative_socre,average_sentence_length,Average_word_length,Avg_Words_Per_Sentence,syllable_count,Complex_word_count,percentage_of_complex_word,Fog_index,Count_personal_pronouns
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,Rise telemedicine Impact Livelihood Blackcoffe...,"[Rise, telemedicine, Impact, Livelihood, Black...",1610,0.080363,0.371509,240,226,16.110000,7,18.811111,4686,872,54.161491,28.108596,21
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,Rise ehealth impact humans year Blackcoffer In...,"[Rise, ehealth, impact, humans, year, Blackcof...",1005,0.106257,0.487765,102,105,28.542857,7,29.485714,2851,522,51.940299,32.193262,21
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,Rise ehealth impact humans year Blackcoffer In...,"[Rise, ehealth, impact, humans, year, Blackcof...",1318,0.104491,0.400472,115,135,15.162791,7,16.305882,3539,620,47.040971,24.881505,24
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,Rise telemedicine Impact Livelihood Blackcoffe...,"[Rise, telemedicine, Impact, Livelihood, Black...",1397,0.109602,0.367165,167,205,17.148148,7,20.857143,3787,684,48.962062,26.444084,23
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,Rise telemedicine Impact Livelihood Blackcoffe...,"[Rise, telemedicine, Impact, Livelihood, Black...",1391,0.111983,0.362926,162,196,18.440000,7,21.043478,3786,694,49.892164,27.332866,23


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   URL_ID                      6 non-null      float64
 1   URL                         6 non-null      object 
 2   Title                       6 non-null      object 
 3   Text                        6 non-null      object 
 4   tokens                      6 non-null      object 
 5   word_count                  6 non-null      int64  
 6   Polarity                    6 non-null      float64
 7   Subjectivity_Score          6 non-null      float64
 8   positive_socre              6 non-null      int64  
 9   negative_socre              6 non-null      int64  
 10  average_sentence_length     6 non-null      float64
 11  Average_word_length         6 non-null      int64  
 12  Avg_Words_Per_Sentence      6 non-null      float64
 13  syllable_count              6 non-null 

In [47]:
df.shape

(6, 18)

In [48]:
#now save the datafrmae to csv file
df.to_csv('result.csv')